<a href="https://colab.research.google.com/github/Baraa710/Premier_League_Predictor/blob/main/BaselineModelAllSeasons.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
seasons = [f'{n}-{n+1}' for n in range(2006,2024)]

In [ ]:

import pandas as pd

dfs = {}
for season in seasons:
    filename = f'/content/drive/MyDrive/APS360_Project/Data/{season}_lessInfo.xlsx'
    df = pd.read_excel(filename, sheet_name='Fixture data')
    dfs[season] = df


In [ ]:
for season, df in dfs.items():
  dfs[season] = dfs[season][["Wk","Home", "Score","Away"]].dropna()

In [ ]:
dfs['2006-2007']

,Wk,Home,Score,Away
0,1.0,Bolton,2–0,Tottenham
1,1.0,Everton,2–1,Watford
2,1.0,Reading,3–2,Middlesbrough
3,1.0,Arsenal,1–1,Aston Villa
4,1.0,West Ham,3–1,Charlton Ath
...,...,...,...,...
425,38.0,Manchester Utd,0–1,West Ham
426,38.0,Tottenham,2–1,Manchester City
427,38.0,Chelsea,1–1,Everton
428,38.0,Portsmouth,0–0,Arsenal


In [ ]:

def get_winner(score):
  home_goals, away_goals = map(int, score.split('–'))
  if home_goals > away_goals:
    return 0
  elif away_goals > home_goals:
    return 1
  else:
    return 2

for season in seasons:
  dfs[season]['winner'] = dfs[season]['Score'].apply(get_winner)
  dfs[season].drop(columns=['Score'], inplace=True)

In [ ]:
for season in seasons:
  dfs[season]['home wins'] = 0
  dfs[season]['away wins'] = 0
  dfs[season]['home games'] = 0
  dfs[season]['away wins'] = 0
  dfs[season].reset_index(inplace=True, drop=True)

In [ ]:
dfs['2006-2007'].loc[0, 'home wins']

0

In [ ]:
season_win_games = {}
for season, df in dfs.items():
  season_win_games[season] = {}
  for team in df['Home'].unique():
    season_win_games[season][team] = (0, 0) # at start of season, 0 wins and 0 games

for season in seasons: # iterate over fixtures from all seasons
  for i, row in dfs[season].iterrows():
    # Write the home/away wine/games values prior to the match represented in the current row
    dfs[season].loc[i, 'home wins'] = season_win_games[season][row['Home']][0]
    dfs[season].loc[i, 'home games'] = season_win_games[season][row['Home']][1]
    dfs[season].loc[i, 'away wins'] = season_win_games[season][row['Away']][0]
    dfs[season].loc[i, 'away games'] = season_win_games[season][row['Away']][1]

    # Write the home/away wine/games based on the result of the current game
    if row['winner'] == 0:
      season_win_games[season][row['Home']] = (season_win_games[season][row['Home']][0] + 1, season_win_games[season][row['Home']][1])
    elif row['winner'] == 1:
      season_win_games[season][row['Away']] = (season_win_games[season][row['Away']][0] + 1, season_win_games[season][row['Away']][1])
    season_win_games[season][row['Home']] = (season_win_games[season][row['Home']][0], season_win_games[season][row['Home']][1] + 1)
    season_win_games[season][row['Away']] = (season_win_games[season][row['Away']][0], season_win_games[season][row['Away']][1] + 1)


In [ ]:
# show all rows in dfs['2008-2009'] where either home or away is Chelsea

chelsea_games = dfs['2007-2008'][ (dfs['2007-2008']['Home']=='Chelsea') | (dfs['2007-2008']['Away']=='Chelsea') ]
chelsea_games


,Wk,Home,Away,winner,home wins,away wins,home games,away games,predicted_winner
9,1.0,Chelsea,Birmingham City,0,0,0,0,0.0,2.0
16,2.0,Reading,Chelsea,1,0,1,1,1.0,1.0
25,3.0,Liverpool,Chelsea,2,1,2,1,2.0,2.0
29,4.0,Chelsea,Portsmouth,0,2,1,3,3.0,0.0
44,5.0,Aston Villa,Chelsea,0,1,3,3,4.0,1.0
50,6.0,Chelsea,Blackburn,2,3,2,5,4.0,0.0
63,7.0,Manchester Utd,Chelsea,0,3,3,6,6.0,2.0
72,8.0,Chelsea,Fulham,2,3,1,7,7.0,0.0
81,9.0,Bolton,Chelsea,1,1,3,8,8.0,1.0
89,10.0,Middlesbrough,Chelsea,1,2,4,9,9.0,1.0


In [ ]:
for season, df in dfs.items():
  for i, row in df.iterrows():
    if df.loc[i, 'home games'] == 0 or df.loc[i, 'away games'] == 0:
      df.loc[i, 'predicted_winner'] = 2
    elif df.loc[i, 'home wins']/df.loc[i, 'home games'] > df.loc[i, 'away wins']/df.loc[i, 'away games']:
      df.loc[i, 'predicted_winner'] = 0
    elif df.loc[i, 'home wins']/df.loc[i, 'home games'] < df.loc[i, 'away wins']/df.loc[i, 'away games']:
      df.loc[i, 'predicted_winner'] = 1
    else:
      df.loc[i, 'predicted_winner'] = 2


In [ ]:
# prompt: find ratio rows where winner and predicted winner are the same

def compare_winner_prediction(df):
  correct_predictions = 0
  for i, row in df.iterrows():
    if row['winner'] == row['predicted_winner']:
      correct_predictions += 1
  return correct_predictions / len(df)

for season, df in dfs.items():
  print(f"Season {season}: {compare_winner_prediction(df):.2f}")


Season 2006-2007: 0.44
Season 2007-2008: 0.51
Season 2008-2009: 0.46
Season 2009-2010: 0.46
Season 2010-2011: 0.41
Season 2011-2012: 0.49
Season 2012-2013: 0.49
Season 2013-2014: 0.52
Season 2014-2015: 0.45
Season 2015-2016: 0.44
Season 2016-2017: 0.52
Season 2017-2018: 0.50
Season 2018-2019: 0.54
Season 2019-2020: 0.47
Season 2020-2021: 0.47
Season 2021-2022: 0.48
Season 2022-2023: 0.48
Season 2023-2024: 0.52


In [ ]:

import pandas as pd
with pd.ExcelWriter('/content/drive/MyDrive/APS360_Project/Match_Data/cleaned_fixture_data.xlsx') as writer:
    for season, df in dfs.items():
        df.to_excel(writer, sheet_name=season)
